#### Load data

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import layers, models, losses, callbacks

from generative_text.general_tnn_generative.process import main, prepare_data
from generative_text.general_tnn_generative.utils.fnProcessing import read_config, view_shapes

import pandas as pd
pd.set_option("display.max_colwidth", None)

config_params = read_config(section='params', config_path='./generative_text/configkeras.ini')
config_clearml = read_config(section='clearml', config_path='./generative_text/configkeras.ini')

'''
Load dataframe with test column
'''
# Sample 
data = pd.read_csv('../data/sampled_data_14.csv')[['thread_id','thread_header','posted_comment','posted_date_time']].sample(1000)
data = prepare_data(data, input_col='posted_comment', clean_col='text')

C:\Users\jwkon\AppData\Local\Temp\ipykernel_30320\2942021222.py:20: DtypeWarning: Columns (3,6,10,11,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/sampled_data_14.csv')[['thread_id','thread_header','posted_comment','posted_date_time']].sample(1000)


#### Context pairing

In [3]:
from generative_text.general_tnn_generative.utils.fnContextPairing import ContextPairing

included_entity_labels = ['PERSON', 'PRODUCT', 'ORG', 'FAC', 'NORP']
# Replace with your path to the dump file. Download here: https://dumps.wikimedia.org/other/wikibase/wikidatawiki/
path_to_dump_file = '../data/simplewiki/simplewiki-20230820-pages-articles-multistream.xml'

data = data.rename(columns={'text_clean': 'text'})
context_pairing = ContextPairing(data, path_to_dump_file, included_entity_labels)
context_pairing.run()

Saved topic data to ./topic
Data saved to ./topic\data_0_topic_986.csv


##### Load context pair data

In [8]:
context_data = pd.read_csv('./topic/context_0_topic_986.csv').dropna()
data = pd.read_csv('./topic/data_0_topic_986.csv')
matched = pd.read_csv('./topic/matched_0_topic_986.csv').dropna()
labels = pd.read_csv('./topic/labels_0_topic_986.txt').dropna()
articles = pd.read_csv('./topic/articles_0_topic_986.csv').dropna()

train_ds, val_ds, test_ds, combined_vocab, tokenizer = main(data, input_col='posted_comment', clean_col='text')

The sample rows below are from `data_0_topic_2432.csv`. I'm unsure of the context for both of these rows, but the first is replying to another post, and the second is a continuation of a previous post. The second contains many keywords, and it appears to be genuine dialog, so I'll use it to demonstrate ContextPairing().

In [ ]:
data.sample(2)

,thread_id,thread_header,posted_comment,posted_date_time,text
51,417814374.0,NaN,"<a href=""#p417814177"" class=""quotelink"">&gt;&gt;417814177</a><br>I agree!",2023-02-27 05:49:01,I agree!
20,420033696.0,NaN,"<a href=""#p420033650"" class=""quotelink"">&gt;&gt;420033650</a><br>Cont.<br><br>This is all happening in the context of a global famine partially triggered by the Ukraine War. Russia, Belarus, and Ukraine happen between them to be the number 1, 2, or 3 global producer of a host of industrial and agricultural inputs, as well as wheat. <br><br>Losing Russia&#039;s supply of natural gas for producing nitrogen fertilizer at the same time as losing Belarus&#039;s supplies of potash and China banning exports of phosphate fertilizers (they are the world&#039;s largest producer) means the third world is going to get wiped out. <br><br>Specifically, most of Russia and Ukraines agricultural export goes to the middle east, which is filled with poor countries that don&#039;t have enough arable land to feed themselves, and most of their fertilizer exports go to Africa, where poor soil means artificial fertilizer is the only thing keeping famine at bay. This is also the case in many Latin American countries. When the price of fertilizer and food goes above a certain point, third worlders will be priced out of the market. <br><br>China is the world&#039;s largest food importer, being hammered by swine flu, rising prices of food and fertilizer, and its own catastrophic water management, while also suffering economic disaster and social upheaval. <br><br>Iran, Iraq, Egypt, the Levant, North Africa, Pakistan, Afghanistan, these countries are going to go to war over food and water and lebensraum in the next year or so, in fact Egypt and Ethiopia already almost went to war over a Nile dam like last year. <br><br>This plane is being crashed with no survivors.",2023-03-17 11:56:52,"Cont. This is all happening in the context of a global famine partially triggered by the Ukraine War. Russia, Belarus, and Ukraine happen between them to be the number 1, 2, or 3 global producer of a host of industrial and agricultural inputs, as well as wheat. Losing Russia's supply of natural gas for producing nitrogen fertilizer at the same time as losing Belarus's supplies of potash and China banning exports of phosphate fertilizers (they are the world's largest producer) means the third world is going to get wiped out. Specifically, most of Russia and Ukraines agricultural export goes to the middle east, which is filled with poor countries that don't have enough arable land to feed themselves, and most of their fertilizer exports go to Africa, where poor soil means artificial fertilizer is the only thing keeping famine at bay. This is also the case in many Latin American countries. When the price of fertilizer and food goes above a certain point, third worlders will be priced out of the market. China is the world's largest food importer, being hammered by swine flu, rising prices of food and fertilizer, and its own catastrophic water management, while also suffering economic disaster and social upheaval. Iran, Iraq, Egypt, the Levant, North Africa, Pakistan, Afghanistan, these countries are going to go to war over food and water and lebensraum in the next year or so, in fact Egypt and Ethiopia already almost went to war over a Nile dam like last year. This plane is being crashed with no survivors."


The training data is `context_data_0_2432.csv`. This will contain the dataframe with the row and matched terms found in the wikidump.

In [ ]:
context_data[context_data['thread_id'] == 420033696.0]

text  \
20                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

#### i. Local Training - Text Generation

#### 1. Local

In [9]:
from generative_text.general_tnn_generative.tnn import TransformerBlock
from generative_text.general_tnn_generative.train import train_model, TrainTextGenerator, CustomSchedule
from generative_text.general_tnn_generative.evaluate import TextGenerator, CustomSchedule

import configparser

config = configparser.ConfigParser()
config.read('./generative_text/configkeras.ini')
params = config["params"]
epochs = int(params['epochs']) 

LOAD_MODEL = False
MODEL_PATH = './models/general_generative/model_1.keras'

if LOAD_MODEL and os.path.exists(MODEL_PATH):
    model = train_model(preload_model=True, model_path=MODEL_PATH)
else:
    model = train_model(preload_model=False, model_path=MODEL_PATH)

def get_callbacks():
    model_checkpoint_callback = ModelCheckpoint(
        filepath='./models/general_generative/weights.{epoch:02d}-{val_loss:.2f}.ckpt',
        save_weights_only=False,
        save_best_only=True,
        monitor='val_loss',                               
        verbose=1)
    text_generator = TrainTextGenerator(index_to_word=combined_vocab,tokenizer=tokenizer)
    early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True)
    return [model_checkpoint_callback, text_generator, early_stopping_callback]

model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=get_callbacks(),
)
model.save(MODEL_PATH)

Epoch 1/150
44/44 [==============================] - ETA: 0s - loss: 5.0537 - dense_21_loss: 5.0537
Epoch 1: val_loss improved from inf to 2.44090, saving model to ./models/general_generative\weights.01-2.44.ckpt


INFO:tensorflow:Assets written to: ./models/general_generative\weights.01-2.44.ckpt\assets


INFO:tensorflow:Assets written to: ./models/general_generative\weights.01-2.44.ckpt\assets


44/44 [==============================] - 11s 185ms/step - loss: 5.0537 - dense_21_loss: 5.0537 - val_loss: 2.4409 - val_dense_21_loss: 2.4409
Epoch 2/150
43/44 [============================>.] - ETA: 0s - loss: 2.0874 - dense_21_loss: 2.0874
Epoch 2: val_loss improved from 2.44090 to 1.76251, saving model to ./models/general_generative\weights.02-1.76.ckpt


INFO:tensorflow:Assets written to: ./models/general_generative\weights.02-1.76.ckpt\assets


...

INFO:tensorflow:Assets written to: ./models/general_generative\weights.33-0.14.ckpt\assets


44/44 [==============================] - 7s 159ms/step - loss: 0.1745 - dense_21_loss: 0.1745 - val_loss: 0.1418 - val_dense_21_loss: 0.1418
Epoch 34/150
44/44 [==============================] - ETA: 0s - loss: 0.2159 - dense_21_loss: 0.2159
Epoch 34: val_loss did not improve from 0.14175
44/44 [==============================] - 3s 60ms/step - loss: 0.2159 - dense_21_loss: 0.2159 - val_loss: 0.1533 - val_dense_21_loss: 0.1533
Epoch 35/150
44/44 [==============================] - ETA: 0s - loss: 0.2251 - dense_21_loss: 0.2251
Epoch 35: val_loss improved from 0.14175 to 0.13858, saving model to ./models/general_generative\weights.35-0.14.ckpt


INFO:tensorflow:Assets written to: ./models/general_generative\weights.35-0.14.ckpt\assets


INFO:tensorflow:Assets written to: ./models/general_generative\weights.35-0.14.ckpt\assets


44/44 [==============================] - 7s 162ms/step - loss: 0.2251 - dense_21_loss: 0.2251 - val_loss: 0.1386 - val_dense_21_loss: 0.1386
Epoch 36/150
44/44 [==============================] - ETA: 0s - loss: 0.1966 - dense_21_loss: 0.1966
Epoch 36: val_loss did not improve from 0.13858
44/44 [==============================] - 3s 57ms/step - loss: 0.1966 - dense_21_loss: 0.1966 - val_loss: 0.1442 - val_dense_21_loss: 0.1442
Epoch 37/150
43/44 [============================>.] - ETA: 0s - loss: 0.2110 - dense_21_loss: 0.2110
Epoch 37: val_loss did not improve from 0.13858
44/44 [==============================] - 3s 60ms/step - loss: 0.2118 - dense_21_loss: 0.2118 - val_loss: 0.2135 - val_dense_21_loss: 0.2135
Epoch 38/150
44/44 [==============================] - ETA: 0s - loss: 0.2204 - dense_21_loss: 0.2204
Epoch 38: val_loss did not improve from 0.13858
44/44 [==============================] - 3s 62ms/step - loss: 0.2204 - dense_21_loss: 0.2204 - val_loss: 0.1796 - val_dense_21_loss:

#### Local Evaluation - Text Generation

Evaluate the text generated by the model using the first row in the sample below. After adding the seed text, I ran the cell three times.

In [15]:
data.sample(2)

,thread_id,thread_header,posted_comment,posted_date_time,text
800,452887098.0,NaN,"<a href=""#p452886245"" class=""quotelink"">&gt;&gt;452886245</a><br>So the Saudi prince was actually the one who did it and the intelligence agencies ran protection for him in order to prevent a war and stop the US from bombing Saudi Arabian oil fields with nukes so they can never be used again?<br>We&#039;ve been living in a machiavellian fanatic/tom clancy nightmare",2023-12-23 23:22:43,So the Saudi prince was actually the one who did it and the intelligence agencies ran protection for him in order to prevent a war and stop the US from bombing Saudi Arabian oil fields with nukes so they can never be used again? We've been living in a machiavellian fanatic/tom clancy nightmare
655,454013599.0,NaN,Also chemtrails aren&#039;t real.,2024-01-04 03:58:23,Also chemtrails aren't real.


In [18]:
from tensorflow.keras.models import load_model
from generative_text.general_tnn_generative.tnn import TransformerBlock, TokenAndPositionEmbedding
from generative_text.general_tnn_generative.evaluate import TextGenerator, CustomSchedule
from tensorflow.keras.utils import custom_object_scope

MODEL_PATH = './models/general_generative/model_1.keras'
with custom_object_scope({'CustomSchedule': CustomSchedule, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding': TokenAndPositionEmbedding}):
    model = load_model(MODEL_PATH)

text_generator = TextGenerator(model, index_to_word=combined_vocab, top_p=0.9)
text = text_generator.generate('So the Saudi prince', max_tokens=15, temperature=0.8)

1/1 [==============================] - 0s 251ms/step


1/1 [==============================] - 0s 21ms/step

generated text:
So the Saudi prince was actually the one who wound it because of qatar buying



#### 2. ClearML - Remote Training - Text Generation

In [ ]:
from clearml import Task, Dataset as ClearMLDataset
from generative_text.general_tnn_generative.process import main
from generative_text.general_tnn_generative.utils.fnProcessing import read_config
from datetime import datetime
import os
import pandas as pd

from generative_text.general_tnn_generative.utils.fnOps import ClearMLOps, ClearMLOpsTraining

def close_task(task_id):
    task = Task.get_task(task_id=task_id)
    task.mark_completed()
    task.close()
    print(f'Task {task_id} closed.')
#close_task('task id')

config_params = read_config(section='params', config_path='./generative_text/configkeras.ini')
clearml_params = read_config(section='clearml', config_path='./generative_text/configkeras.ini')
clearml_project_name = clearml_params['clearml_project_name']
clearml_model_id = clearml_params['clearml_model_id']
clearml_output_uri = clearml_params['clearml_output_uri']
model_name = clearml_params['model_name']
model_path = clearml_params['model_path']

# Initialize ClearMLOpsTraining with both params
clean_training = ClearMLOpsTraining(clearml_params, config_params)
clean_ops = ClearMLOps(config_path='./generative_text/configkeras.ini')

clean_ops.set_creds_connect()

In [ ]:
clean_ops.process_data(sampled,'dataset_name','task name')

In [ ]:
# End each task with number - contextdialog1
# No spaces - contextdialog1_rt1wor

task_name = 'task name'
model = clean_training.run_clearml_training_task(task_name, dataset_id='dataset_id', load_model='train', model_id='model_id')

#### Remote Evaluation - Text Generation

In [ ]:
from generative_text.general_tnn_generative.tnn import TransformerBlock, TokenAndPositionEmbedding
from generative_text.general_tnn_generative.evaluate import TextGenerator, CustomSchedule
from tensorflow.keras.utils import custom_object_scope

MODEL_PATH = 'model_path'
with custom_object_scope({'CustomSchedule': CustomSchedule, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding': TokenAndPositionEmbedding}):
    clear_ml_model = load_model(MODEL_PATH)
text_generator = TextGenerator(clear_ml_model, index_to_word=vocab)
text = text_generator.generate('Add your seed', max_tokens=200, temperature=0.9)